# OmniSafe Tutorial - Environment Customization From Scratch

OmniSafe: https://github.com/PKU-Alignment/omnisafe

Documentation: https://omnisafe.readthedocs.io/en/latest/

Safety-Gymnasium: https://www.safety-gymnasium.com/

[Safety-Gymnasium](https://www.safety-gymnasium.com/) is a highly scalable and customizable Safe Reinforcement Learning library, aiming to deliver a good view of benchmarking Safe Reinforcement Learning (Safe RL) algorithms and a more standardized setting of environments. 

## 引言

本节与[Tutorial 4: Environment Customization from Community](./4.Environment%20Customization%20from%20Community.ipynb)共同介绍了如何令定制化环境享受OmniSafe提供的全套训练、记录与保存框架。本节侧重于面向安全强化学习初学者介绍如何从零开始创建环境；而[Tutorial 4: Environment Customization from Community](./4.Environment%20Customization%20from%20Community.ipynb)关注如何将社区已有的环境，例如[Gymnasium](https://github.com/Farama-Foundation/Gymnasium)，作出最小适配，以嵌入OmniSafe中。

具体而言，本节提供了一个用于定制化环境的最简单模版。通过该模版，您将了解：

- 如何在OmniSafe中创建并注册一个环境。
- 如何指定创建环境时的定制化参数。
- 如何记录环境特定的信息。

## 快速安装

In [ ]:
# 通过pip安装（如果您已经安装，请忽略此段代码）
!pip install omnisafe

In [ ]:
# 通过源代码安装（如果您已经安装，请忽略此段代码）
## 克隆仓库
!git clone https://github.com/PKU-Alignment/omnisafe
%cd omnisafe

## 完成安装
!pip install -e .

## 定制化环境最简模版
OmniSafe的定制化环境可以仅通过单个文件实现。我们将为您介绍一个最简的定制化环境模版，它将作为您入门的起点。

### 定制化环境设计
我们将在此细致地介绍一个简易随机环境的设计过程。如果您是强化学习领域的专家或有经验的研究者，可以跳过该模块至[定制化环境嵌入](#定制化环境嵌入)或[Tutorial 4: Environment Customization from Community](./4.Gymnasium%20Customization.ipynb)。

In [1]:
# 导入必要的包
from __future__ import annotations

import random
import omnisafe
from typing import Any, ClassVar

import torch
from gymnasium import spaces

from omnisafe.envs.core import CMDP, env_register, env_unregister

In [2]:
# 定义环境类
class ExampleEnv(CMDP):
    _support_envs: ClassVar[list[str]] = ['Example-v0']  # 受支持的任务名称

    need_auto_reset_wrapper = True  # 是否需要 `AutoReset` Wrapper
    need_time_limit_wrapper = True  # 是否需要 `TimeLimit` Wrapper

您需要关注上面这段代码的如下细节：

- **任务名称定义** 在 `_support_envs`中提供环境受支持的任务名称。
- **Wrapper配置** 通过设定 `need_auto_reset_wrapper`和 `need_time_limit_wrapper` 来定义自动重置和限制时间。
- **并行环境数量** 如果您的环境支持向量化并行，请通过 `_num_envs` 参数进行设定。

In [3]:
class ExampleEnv(CMDP):
    _support_envs: ClassVar[list[str]] = ['Example-v0', 'Example-v1']  # 受支持的任务名称

    need_auto_reset_wrapper = True  # 是否需要 `AutoReset` Wrapper
    need_time_limit_wrapper = True  # 是否需要 `TimeLimit` Wrapper

    def __init__(self, env_id: str, **kwargs) -> None:
        self._count = 0
        self._num_envs = 1
        self._observation_space = spaces.Box(low=-1.0, high=1.0, shape=(3,))
        self._action_space = spaces.Box(low=-1.0, high=1.0, shape=(2,))

完成 `__init__`函数定义。此处需要给出环境的动作空间与观测空间。您需要根据您当前在设计的具体任务来定义。例如：
```python
if env_id == 'Example-v0':
    self._observation_space = spaces.Box(low=-1.0, high=1.0, shape=(3,))
    self._action_space = spaces.Box(low=-1.0, high=1.0, shape=(2,))
elif env_id == 'Example-v1':
    self._observation_space = spaces.Box(low=-1.0, high=1.0, shape=(4,))
    self._action_space = spaces.Box(low=-1.0, high=1.0, shape=(3,))
else:
    raise NotImplementedError
```
**请注意：** 由于需要为上层模块提供标准的接口，因此在设计环境时请遵循 `self._observation_space` 以及 `self._action_space` 这两个变量名**

完成环境初始化相关函数的定义。`reset` 和 `set_seed` 是OmniSafe环境初始化的标准接口。其中 `reset` 重置环境状态与计步器。 `set_seed` 通过设定随机种子确保实验的可复现性。而带有`@property`装饰器的`max_episode_steps`函数用于为`TimeLimit` Wrapper传递需要限制的每幕最大步数。实现参考如下：

In [4]:
class ExampleEnv(CMDP):
    _support_envs: ClassVar[list[str]] = ['Example-v0', 'Example-v1']  # 受支持的任务名称

    need_auto_reset_wrapper = True  # 是否需要 `AutoReset` Wrapper
    need_time_limit_wrapper = True  # 是否需要 `TimeLimit` Wrapper

    def __init__(self, env_id: str, **kwargs) -> None:
        self._count = 0
        self._num_envs = 1
        self._observation_space = spaces.Box(low=-1.0, high=1.0, shape=(3,))
        self._action_space = spaces.Box(low=-1.0, high=1.0, shape=(2,))

    def set_seed(self, seed: int) -> None:
        random.seed(seed)

    def reset(
        self,
        seed: int | None = None,
        options: dict[str, Any] | None = None,
    ) -> tuple[torch.Tensor, dict]:
        if seed is not None:
            self.set_seed(seed)
        obs = torch.as_tensor(self._observation_space.sample())
        self._count = 0
        return obs, {}

    @property
    def max_episode_steps(self) -> None:
        """The max steps per episode."""
        return 10

完成功能性函数的定义。`render` 函数用于渲染环境；`close` 函数用于训练结束后的清理。

In [5]:
class ExampleEnv(CMDP):
    _support_envs: ClassVar[list[str]] = ['Example-v0', 'Example-v1']  # 受支持的任务名称

    need_auto_reset_wrapper = True  # 是否需要 `AutoReset` Wrapper
    need_time_limit_wrapper = True  # 是否需要 `TimeLimit` Wrapper

    def __init__(self, env_id: str, **kwargs) -> None:
        self._count = 0
        self._num_envs = 1
        self._observation_space = spaces.Box(low=-1.0, high=1.0, shape=(3,))
        self._action_space = spaces.Box(low=-1.0, high=1.0, shape=(2,))

    def set_seed(self, seed: int) -> None:
        random.seed(seed)

    def reset(
        self,
        seed: int | None = None,
        options: dict[str, Any] | None = None,
    ) -> tuple[torch.Tensor, dict]:
        if seed is not None:
            self.set_seed(seed)
        obs = torch.as_tensor(self._observation_space.sample())
        self._count = 0
        return obs, {}

    @property
    def max_episode_steps(self) -> None:
        """The max steps per episode."""
        return 10

    def render(self) -> Any:
        pass

    def close(self) -> None:
        pass

完成 `step` 函数定义。此处是您定制化环境的核心交互逻辑。您只需按照本例中的数据输入与输出格式进行调整即可。您也可以直接将本例中的随机交互动态更改为您的环境动态。

In [6]:
class ExampleEnv(CMDP):
    _support_envs: ClassVar[list[str]] = ['Example-v0', 'Example-v1']  # 受支持的任务名称
    metadata: ClassVar[dict[str, int]] = {}

    need_auto_reset_wrapper = True  # 是否需要 `AutoReset` Wrapper
    need_time_limit_wrapper = True  # 是否需要 `TimeLimit` Wrapper

    def __init__(self, env_id: str, **kwargs) -> None:
        self._count = 0
        self._num_envs = 1
        self._observation_space = spaces.Box(low=-1.0, high=1.0, shape=(3,))
        self._action_space = spaces.Box(low=-1.0, high=1.0, shape=(2,))

    def set_seed(self, seed: int) -> None:
        random.seed(seed)

    def reset(
        self,
        seed: int | None = None,
        options: dict[str, Any] | None = None,
    ) -> tuple[torch.Tensor, dict]:
        if seed is not None:
            self.set_seed(seed)
        obs = torch.as_tensor(self._observation_space.sample())
        self._count = 0
        return obs, {}

    @property
    def max_episode_steps(self) -> None:
        """The max steps per episode."""
        return 10

    def render(self) -> Any:
        pass

    def close(self) -> None:
        pass

    def step(
        self,
        action: torch.Tensor,
    ) -> tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor, dict]:
        self._count += 1
        obs = torch.as_tensor(self._observation_space.sample())
        reward = 2 * torch.as_tensor(random.random())
        cost = 2 * torch.as_tensor(random.random())
        terminated = torch.as_tensor(random.random() > 0.9)
        truncated = torch.as_tensor(self._count > 10)
        return obs, reward, cost, terminated, truncated, {'final_observation': obs}

接下来，我们试着运行该环境10个时间步，观察交互信息。

In [7]:
env = ExampleEnv(env_id='Example-v0')
env.reset(seed=0)
while True:
    action = env.action_space.sample()
    obs, reward, cost, terminated, truncated, info = env.step(action)
    print('-' * 20)
    print(f'obs: {obs}')
    print(f'reward: {reward}')
    print(f'cost: {cost}')
    print(f'terminated: {terminated}')
    print(f'truncated: {truncated}')
    print('*' * 20)
    if terminated or truncated:
        break
env.close()

--------------------
obs: tensor([-0.5552,  0.2905,  0.0094])
reward: 1.6888437271118164
cost: 1.5159088373184204
terminated: False
truncated: False
********************
--------------------
obs: tensor([-0.0635, -0.9966, -0.4681])
reward: 0.5178334712982178
cost: 1.0225493907928467
terminated: False
truncated: False
********************
--------------------
obs: tensor([ 0.4385,  0.0678, -0.3470])
reward: 1.5675971508026123
cost: 0.6066254377365112
terminated: False
truncated: False
********************
--------------------
obs: tensor([ 0.8278, -0.5252, -0.1799])
reward: 1.1667640209197998
cost: 1.8162257671356201
terminated: False
truncated: False
********************
--------------------
obs: tensor([ 0.1086, -0.5711,  0.7751])
reward: 0.5636757016181946
cost: 1.511608362197876
terminated: False
truncated: False
********************
--------------------
obs: tensor([-0.3585,  0.8011,  0.2172])
reward: 0.5010126829147339
cost: 1.8194924592971802
terminated: True
truncated: False
***

恭喜您！已经成功完成了基础的环境定义，接下来，我们将介绍如何将该环境注册入OmniSafe中，并实现环境参数传递、交互信息记录、算法训练以及结果保存等步骤。

### 定制化环境嵌入

### 快速训练

得益于OmniSafe精心设计的注册机制，我们只需一个装饰器即可将这个环境注册到OmniSafe的环境列表中。

In [8]:
@env_register
class ExampleEnv(ExampleEnv):
    pass

注册同名环境将会报错，这是由于**环境名称冲突**。

In [9]:
@env_register
class CustomExampleEnv(ExampleEnv):
    example_configs = 1


env = CustomExampleEnv('Example-v0')
env.example_configs

1

这时，您需要先对环境手动取消注册。

In [10]:
@env_unregister
class CustomExampleEnv(ExampleEnv):
    pass

之后，您就可以重新注册该环境了。在本教程中，我们会同时嵌套 `env_register` 和 `env_unregister` 装饰器，这是为了避免环境重复注册造成报错，即确保该环境只被注册一次，以便用户在阅读本教程时多次修改与运行代码。

In [11]:
@env_register
@env_unregister
class CustomExampleEnv(ExampleEnv):
    example_configs = 2


env = CustomExampleEnv('Example-v0')
env.example_configs

CustomExampleEnv has not been registered yet


2

随后，您可以使用OmniSafe中的算法来训练这个自定义环境。

In [12]:
custom_cfgs = {
    'train_cfgs': {
        'total_steps': 30,
    },
    'algo_cfgs': {
        'steps_per_epoch': 10,
        'update_iters': 1,
    },
}
agent = omnisafe.Agent('PPOLag', 'Example-v0', custom_cfgs=custom_cfgs)
agent.learn()

Loading PPOLag.yaml from /home/safepo/dev-env/omnisafe_zjy/omnisafe/utils/../configs/on-policy/PPOLag.yaml


Logging data to ./runs/PPOLag-{Example-v0}/seed-000-2024-04-09-15-04-56/progress.csv

Save with config in config.json

INFO: Start training

/home/safepo/anaconda3/envs/dev-env/lib/python3.8/site-packages/rich/live.py:231: UserWarning: install "ipywidgets"
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

...

Warning: trajectory cut off when rollout by epoch at 10.0 steps.

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Metrics                        ┃ Value                  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Metrics/EpRet                  │ 7.8531365394592285     │
│ Metrics/EpCost                 │ 7.931504726409912      │
│ Metrics/EpLen                  │ 6.666666507720947      │
│ Train/Epoch                    │ 1.0                    │
│ Train/Entropy                  │ 1.4192386865615845     │
│ Train/KL                       │ 6.416345422621816e-05  │
│ Train/StopIter                 │ 1.0                    │
│ Train/PolicyRatio/Mean         │ 1.0                    │
│ Train/PolicyRatio/Min          │ 1.0                    │
│ Train/PolicyRatio/Max          │ 1.0                    │
│ Train/PolicyRatio/Std          │ 0.0                    │
│ Train/LR                       │ 9.999999747378752e-05  │
│ Train/PolicyStd                │ 1.0003000497817993     │
│ TotalEnvSteps                  │ 20.0                   │
│ Loss/Loss_pi                   │ -6.258487417198921e-08 │
│ Loss/Loss_pi/Delta             │ -4.768371297814156e-08 │
│ Value/Adv                      │ 1.341104507446289e-07  │
│ Loss/Loss_reward_critic        │ 38.05686950683594      │
│ Loss/Loss_reward_critic/Delta  │ 27.59790325164795      │
│ Value/reward                   │ -0.008213319815695286  │
│ Loss/Loss_cost_critic          │ 23.737285614013672     │
│ Loss/Loss_cost_critic/Delta    │ 4.595714569091797      │
│ Value/cost                     │ 0.17113244533538818    │
│ Time/Total                     │ 0.0776519775390625     │
│ Time/Rollout                   │ 0.015673398971557617   │
│ Time/Update                    │ 0.011301994323730469   │
│ Time/Epoch                     │ 0.027007579803466797   │
│ Time/FPS                       │ 370.27294921875        │
│ Metrics/LagrangeMultiplier/Mea │ 0.0                    │
│ Metrics/LagrangeMultiplier/Min │ 0.0                    │
│ Metrics/LagrangeMultiplier/Max │ 0.0                    │
│ Metrics/LagrangeMultiplier/Std │ 0.0                    │
└────────────────────────────────┴────────────────────────┘

Warning: trajectory cut off when rollout by epoch at 9.0 steps.

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Metrics                        ┃ Value                   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Metrics/EpRet                  │ 6.297085762023926       │
│ Metrics/EpCost                 │ 6.2187700271606445      │
│ Metrics/EpLen                  │ 5.25                    │
│ Train/Epoch                    │ 2.0                     │
│ Train/Entropy                  │ 1.419387698173523       │
│ Train/KL                       │ 5.490810053743189e-06   │
│ Train/StopIter                 │ 1.0                     │
│ Train/PolicyRatio/Mean         │ 1.0                     │
│ Train/PolicyRatio/Min          │ 1.0                     │
│ Train/PolicyRatio/Max          │ 1.0                     │
│ Train/PolicyRatio/Std          │ 0.0                     │
│ Train/LR                       │ 0.0                     │
│ Train/PolicyStd                │ 1.0004494190216064      │
│ TotalEnvSteps                  │ 30.0                    │
│ Loss/Loss_pi                   │ -1.9073486612342094e-07 │
│ Loss/Loss_pi/Delta             │ -1.2814999195143173e-07 │
│ Value/Adv                      │ 1.0728835775353218e-07  │
│ Loss/Loss_reward_critic        │ 34.77037811279297       │
│ Loss/Loss_reward_critic/Delta  │ -3.2864913940429688     │
│ Value/reward                   │ 0.014150517992675304    │
│ Loss/Loss_cost_critic          │ 27.43436050415039       │
│ Loss/Loss_cost_critic/Delta    │ 3.6970748901367188      │
│ Value/cost                     │ 0.24021005630493164     │
│ Time/Total                     │ 0.12173724174499512     │
│ Time/Rollout                   │ 0.01879405975341797     │
│ Time/Update                    │ 0.011112689971923828    │
│ Time/Epoch                     │ 0.0299375057220459      │
│ Time/FPS                       │ 334.039794921875        │
│ Metrics/LagrangeMultiplier/Mea │ 0.0                     │
│ Metrics/LagrangeMultiplier/Min │ 0.0                     │
│ Metrics/LagrangeMultiplier/Max │ 0.0                     │
│ Metrics/LagrangeMultiplier/Std │ 0.0                     │
└────────────────────────────────┴─────────────────────────┘

(6.297085762023926, 6.2187700271606445, 5.25)

干得不错！我们已经完成了这个定制化环境的嵌入和训练。接下来，我们将进一步研究如何为环境指定超参数。

### 参数设定

我们从一个新的示例环境出发，假设这个环境需要传入一个名为 `num_agents` 的参数。我们将展示如何不修改OmniSafe的代码来完成参数设定。

In [13]:
@env_register
@env_unregister
class NewExampleEnv(ExampleEnv):  # 创造一个新环境
    _support_envs: ClassVar[list[str]] = ['NewExample-v0', 'NewExample-v1']
    num_agents: ClassVar[int] = 1

    def __init__(self, env_id: str, **kwargs) -> None:
        super(NewExampleEnv, self).__init__(env_id, **kwargs)
        self.num_agents = kwargs.get('num_agents', 1)

NewExampleEnv has not been registered yet


此时，`num_agents` 参数为预设值：`1`。

In [14]:
new_env = NewExampleEnv('NewExample-v0')
new_env.num_agents

1

下面我们将展示如何通过 OmniSafe 的接口对该参数进行修改并训练：

In [15]:
custom_cfgs.update({'env_cfgs': {'num_agents': 2}})
agent = omnisafe.Agent('PPOLag', 'NewExample-v0', custom_cfgs=custom_cfgs)
agent.agent._env._env.num_agents

Loading PPOLag.yaml from /home/safepo/dev-env/omnisafe_zjy/omnisafe/utils/../configs/on-policy/PPOLag.yaml


Logging data to ./runs/PPOLag-{NewExample-v0}/seed-000-2024-04-09-15-05-09/progress.csv

Save with config in config.json

2

非常好！我们将 `num_agents` 设置为了2。这表示我们在未修改代码的情形下成功实现了超参数设定。

### 训练信息记录

在运行训练代码时，您可能已经发现 OmniSafe 通过 `Logger` 记录了训练信息，例如：

```bash
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Metrics                        ┃ Value                   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Metrics/EpRet                  │ 2.046875                │
│ Metrics/EpCost                 │ 2.89453125              │
│ Metrics/EpLen                  │ 3.25                    │
│ Train/Epoch                    │ 3.0                     │
...
```
那么我们可否将环境之中的信息输出到日志中呢？答案是肯定的，而且这个过程同样不需要修改OmniSafe的代码。只需要实现两个标准接口：
1. 在 `__init__` 函数中，将需要输出的信息添加到`self.env_spec_log`中。
2. 实例化 `spec_log` 函数，记录所需的信息。

**请注意：** 目前OmniSafe仅支持在每一个epoch结束时记录这些信息，而不支持在每一个step结束时记录。

In [16]:
@env_register
@env_unregister
class NewExampleEnv(ExampleEnv):
    _support_envs: ClassVar[list[str]] = ['NewExample-v0', 'NewExample-v1']

    # 定义需要记录的信息
    def __init__(self, env_id: str, **kwargs) -> None:
        super(NewExampleEnv, self).__init__(env_id, **kwargs)
        self.env_spec_log = {'Env/Success_counts': 0}

    # 通过step函数，与环境进行交互
    def step(self, action):
        obs, reward, cost, terminated, truncated, info = super().step(action)
        success = int(reward > cost)
        self.env_spec_log['Env/Success_counts'] += success
        return obs, reward, cost, terminated, truncated, info

    # 在logger中记录环境信息
    def spec_log(self, logger) -> dict[str, Any]:
        logger.store({'Env/Success_counts': self.env_spec_log['Env/Success_counts']})
        self.env_spec_log['Env/Success_counts'] = 0

接下来，我们简单训练观察该信息是否被成功记录。

In [17]:
custom_cfgs.update({'train_cfgs': {'total_steps': 10}})
agent = omnisafe.Agent('PPOLag', 'NewExample-v0', custom_cfgs=custom_cfgs)
agent.learn()

Loading PPOLag.yaml from /home/safepo/dev-env/omnisafe_zjy/omnisafe/utils/../configs/on-policy/PPOLag.yaml


Logging data to ./runs/PPOLag-{NewExample-v0}/seed-000-2024-04-09-15-05-14/progress.csv

Save with config in config.json

INFO: Start training

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Metrics                        ┃ Value                  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Metrics/EpRet                  │ 5.625942230224609      │
│ Metrics/EpCost                 │ 6.960921287536621      │
│ Metrics/EpLen                  │ 5.0                    │
│ Train/Epoch                    │ 0.0                    │
│ Train/Entropy                  │ 1.4189385175704956     │
│ Train/KL                       │ 0.00024281258811242878 │
│ Train/StopIter                 │ 1.0                    │
│ Train/PolicyRatio/Mean         │ 1.0                    │
│ Train/PolicyRatio/Min          │ 1.0                    │
│ Train/PolicyRatio/Max          │ 1.0                    │
│ Train/PolicyRatio/Std          │ 0.0                    │
│ Train/LR                       │ 0.0                    │
│ Train/PolicyStd                │ 1.0                    │
│ TotalEnvSteps                  │ 10.0                   │
│ Loss/Loss_pi                   │ -5.662441182607836e-08 │
│ Loss/Loss_pi/Delta             │ -5.662441182607836e-08 │
│ Value/Adv                      │ 1.2814999195143173e-07 │
│ Loss/Loss_reward_critic        │ 10.477845191955566     │
│ Loss/Loss_reward_critic/Delta  │ 10.477845191955566     │
│ Value/reward                   │ -0.0091781010851264    │
│ Loss/Loss_cost_critic          │ 18.525999069213867     │
│ Loss/Loss_cost_critic/Delta    │ 18.525999069213867     │
│ Value/cost                     │ 0.14141643047332764    │
│ Time/Total                     │ 0.030597209930419922   │
│ Time/Rollout                   │ 0.017596960067749023   │
│ Time/Update                    │ 0.012219905853271484   │
│ Time/Epoch                     │ 0.02985072135925293    │
│ Time/FPS                       │ 335.00830078125        │
│ Env/Success_counts             │ 1.5                    │
│ Metrics/LagrangeMultiplier/Mea │ 0.0                    │
│ Metrics/LagrangeMultiplier/Min │ 0.0                    │
│ Metrics/LagrangeMultiplier/Max │ 0.0                    │
│ Metrics/LagrangeMultiplier/Std │ 0.0                    │
└────────────────────────────────┴────────────────────────┘

(5.625942230224609, 6.960921287536621, 5.0)

漂亮！上述代码将在终端输出了环境特化的信息 `Env/Success_counts`。这一过程并不需要对原代码作出改动。

## 总结
OmniSafe旨在成为安全强化学习的基础软件。我们将持续完善OmniSafe的环境接口标准，使OmniSafe能够适应各种安全强化学习任务，赋能多元安全场景。